# Project 2 Data sources:

## Observed dependent variable:
* Drug-induced mortality per county for 2016
* reach: opioid-induced mortality per county for 2016
* double-reach: opioid-induced mortality per county over 5 years, 2011 - 2016

## Independent variables/regressors:

### Medical:
* Age of electronic PDMP per state
* Number of methadone clinics per county
* Prescribing map -> rate of opioid prescriptions per 100 US residents

### Legislative:
* Number of bills per state that contain the word 'opioid'

### Economic:
* Poverty
* Percent unemployment
* Median salary

### Demographic:
* Median age
* Percentage minority
* Education - percentage high school grad or higher

Websites for data sources:
* https://wonder.cdc.gov/
* https://www.bls.gov/lau/data.htm
* https://censusreporter.org/

## Observed dependent variable

In order to get the data for the observed dependent variable, it will be necessary to use Selenium and chromedriver to get these tables from CDC Wonder.

In [27]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import pickle
import time

from bs4 import BeautifulSoup

In [28]:
with open("county_df.pkl", 'rb') as picklefile: # again, rb stands for read binary
    [county_df, states] = pickle.load(picklefile)

In [29]:
states[11] = "Hawaii"
states_by_five = [list(states[i:i+5]) for i in range(0, len(states), 5)]

states_by_five

[['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California'],
 ['Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida'],
 ['Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana'],
 ['Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine'],
 ['Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi'],
 ['Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire'],
 ['New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota'],
 ['Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island'],
 ['South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah'],
 ['Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin'],
 ['Wyoming']]

In [30]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time

import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

cdc_wonder_mcd = "https://wonder.cdc.gov/mcd-icd10.html"
driver.get(cdc_wonder_mcd)

In [31]:
agree_button = driver.find_element_by_name("action-I Agree")
agree_button.click()
time.sleep(2)

In [32]:
# Select group by option for results as county
select_group_results = Select(driver.find_element_by_id("SB_1"))
select_group_results.select_by_visible_text("County")

In [33]:
# Check box to get age-adjusted mortality rate
AAR_checkbox = driver.find_element_by_id("CO_aar_enable_label")
AAR_checkbox.click()

In [34]:
def search_dict_values(dic, search_for):
    for key, value in dic.items():
        if search_for in value:
            return key

In [35]:
# Select location. Sometimes it works if all the states are selected, but often the request times out.
# Will select Alabama first.
select_location = Select(driver.find_element_by_id("codes-D77.V9"))

location_dict = {}
for option in select_location.options:
    location_dict[option.get_attribute('value')] = option.text

# let's get the location values for the first 5 states.
location_values = [search_dict_values(location_dict, state) for state in states_by_five[0]]
#location_value = search_dict_values(location_dict, "Alabama")
for location_value in location_values:
    select_location.select_by_value(location_value)

select_location.deselect_by_value("*All*")

In [36]:
# Select year and month; pick 2016
select_year = Select(driver.find_element_by_id("codes-D77.V1"))
select_year.select_by_value("2016")
select_year.deselect_by_value("*All*")

In [37]:
# For underlying cause of death, click radio button for UCD - Drug/Alcohol Induced Causes
drug_alcohol_causes = driver.find_element_by_id("RO_ucdD77.V25")
drug_alcohol_causes.click()

In [38]:
# Now, select all drug induced causes of death.
select_drug_causes = Select(driver.find_element_by_id("codes-D77.V25"))

#for option in select_drug_causes.options:
#    print(option.text)
#    print(option.get_attribute('value'))

select_drug_causes.select_by_value("D")
select_drug_causes.deselect_by_value("*All*")

In [39]:
# Submit this request to the last send button on the page, so that all of the criteria are included.
send_button = driver.find_element_by_xpath("//div[@class='footer-buttons']/input")
#print(send_button.get_attribute('title'))
send_button.click()

# It takes time for CDC wonder to answer the query and display the results page.
time.sleep(10)

In [17]:
def get_od_mort_df(driver):

# Get the page of results, then get all the tables in the page.
# Get all elements tagged <tr>, and then find the table rows of interest by finding rows containing th class='v'.
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    table = soup.select("tbody")
    table_string = str(table[0])
    table_string_split = table_string.split("<tr>")

    rows = []

    for table_row in table_string_split:
        if 'th class="v"' in table_row:
            rows.append(table_row)

# Split each row by the <td> tag to get a list of each element in the row.
    split_rows = [row.split("<td>") for row in rows]

# Use regular expressions to get rid of all html tags (everything in <>), also commas, tabs and newlines.
    regex = r'(\<.*?\>)|([\n\t,]*)'

    for index, row in enumerate(split_rows):
        split_rows[index] = [re.sub(regex, "", cell) for cell in row]

    cols = ['county_name', 'overdose_deaths', 'population', 'crude_rate', 'age_adj_rate']
    od_mort_df = pd.DataFrame(split_rows, columns=cols)

# The INCITS is located in parentheses at the end of the county_name. It will be easier to join this dataframe
# to the overall dataframe by joining on INCIT, so we'll go ahead and get the INCITS for each row, create a new
# column called INCITS, and then drop the county_name.
    county_names = list(od_mort_df.county_name)
    incits = [county_name.split(" ")[-1] for county_name in county_names]
    incits = [re.sub(r'\(*\)*', "", incit) for incit in incits]
    od_mort_df['INCITS'] = incits
    
    return od_mort_df

In [18]:
od_mort_df = get_od_mort_df(driver)

In [19]:
# After you are done parsing the results, then you can click back to the request page in order to
# re-run the request, except this time you are going to pick a different set of states, and also
# de-select the states that you just queried. Query 5 states at a time.
request_tab = driver.find_element_by_name("tab-request")
request_tab.click()
time.sleep(5)

## Start loop to finish queries for all states

Now that the appropriate dataframe has been started, we can loop through states_by_five and finish the queries for all of the other 45 states. We will temporarily store the results for these tables in a temporary dataframe, and then concatenate it to the original od_mort_df.

In [20]:
location_dict

{'*All*': '   *All*  (The United States)',
 '01': '+ 01  (Alabama)  ',
 '02': '+ 02  (Alaska)  ',
 '04': '+ 04  (Arizona)  ',
 '05': '+ 05  (Arkansas)  ',
 '06': '+ 06  (California)  ',
 '08': '+ 08  (Colorado)  ',
 '09': '+ 09  (Connecticut)  ',
 '10': '+ 10  (Delaware)  ',
 '11': '+ 11  (District of Columbia)  ',
 '12': '+ 12  (Florida)  ',
 '13': '+ 13  (Georgia)  ',
 '15': '+ 15  (Hawaii)  ',
 '16': '+ 16  (Idaho)  ',
 '17': '+ 17  (Illinois)  ',
 '18': '+ 18  (Indiana)  ',
 '19': '+ 19  (Iowa)  ',
 '20': '+ 20  (Kansas)  ',
 '21': '+ 21  (Kentucky)  ',
 '22': '+ 22  (Louisiana)  ',
 '23': '+ 23  (Maine)  ',
 '24': '+ 24  (Maryland)  ',
 '25': '+ 25  (Massachusetts)  ',
 '26': '+ 26  (Michigan)  ',
 '27': '+ 27  (Minnesota)  ',
 '28': '+ 28  (Mississippi)  ',
 '29': '+ 29  (Missouri)  ',
 '30': '+ 30  (Montana)  ',
 '31': '+ 31  (Nebraska)  ',
 '32': '+ 32  (Nevada)  ',
 '33': '+ 33  (New Hampshire)  ',
 '34': '+ 34  (New Jersey)  ',
 '35': '+ 35  (New Mexico)  ',
 '36': '+ 36  (Ne

In [21]:
for i in range(1, len(states_by_five)):

    select_location = Select(driver.find_element_by_id("codes-D77.V9"))

    # deselect all the previous location values
    for location_value in location_values:
        select_location.deselect_by_value(location_value)

    # Get a new set of location values.
    location_values = [search_dict_values(location_dict, state) for state in states_by_five[i]]

    for location_value in location_values:
        select_location.select_by_value(location_value)
        
    # Submit this request to the last send button on the page, so that all of the criteria are included.
    send_button = driver.find_element_by_xpath("//div[@class='footer-buttons']/input")
    send_button.click()
    
    time.sleep(10)
    
    temp_od_mort_df = get_od_mort_df(driver)
    
    od_mort_df = pd.concat([od_mort_df, temp_od_mort_df], ignore_index=True)
    
    # After you are done parsing the results, then you can click back to the request page in order to
    # re-run the request, except this time you are going to pick a different set of states, and also
    # de-select the states that you just queried. Query 5 states at a time.
    request_tab = driver.find_element_by_name("tab-request")
    request_tab.click()
    time.sleep(5)

In [23]:
driver.close()

In [25]:
od_mort_df.head()

,county_name,overdose_deaths,population,crude_rate,age_adj_rate,INCITS
0,Baldwin County AL (01003),34,208563,16.3,15.7,01003
1,Blount County AL (01009),15,57704,Unreliable,Unreliable,01009
2,Calhoun County AL (01015),16,114611,Unreliable,Unreliable,01015
3,Cullman County AL (01043),20,82471,24.3,26.4,01043
4,DeKalb County AL (01049),19,70900,Unreliable,Unreliable,01049


In [26]:
# save od_mort_df dataframe to pickle.
with open("od_mort_df.pkl", "wb") as picklefile:
    pickle.dump(od_mort_df, picklefile)

In [40]:
od_mort_df.shape

(1000, 6)